To be clear, here we are examining instance properties. That is, we define the property in the class we are defining, but the property itself is going to be instance specific, i.e. different instances will support different values for the property. Just like instance attributes. The main difference is that we will use accessor method to get, set (and optionally) delete the associated instance value.

As I mentioned in the lecture, because properties use the same dotted notation (and the same getattr, setattr and delattr functions), we do not need to start with properties. Often a bare attribute works just fine, and if, later, we decide we need to manage getting/setting/deleting the attribute value, we can switch over to properties without breaking our class interface. This is unlike languages like Java - and hence why in those languages it is recommended to always use getter and setter functions. Not so in Python!

A <b>property</b> in Python is essentially a class instance - we'll come back to what that class looks like when we study descriptors. For now, we are going to use the property function in Python which is a convenience callable essentially.

Let's start with a simple example and a bare attribute:

In [1]:
class Person:
    def __init__(self,name):
        self.name = name

In [2]:
p = Person('Alex')

In [3]:
p.name


'Alex'

In [4]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [5]:
p.__dict__

{'name': 'Alex'}

In [6]:
getattr(p, 'name'), setattr(p, 'name', 'Eric')

('Alex', None)

In [7]:
p.name


'Eric'

In [9]:
class Person:
    def __init__(self,name):
        self._name = name
        
    def getname(self):
        return self._name
    
    def setname(self,value):
        if isinstance(value,str) and len(value.strip())>0:
            self._name = value
        else:
            raise ValueError('name must be a non empty string')

In [11]:
p = Person('Alex')

In [19]:
import traceback
try:
    p.setname(100)
except ValueError as ex:
    print(ex, traceback.format_exc())

name must be a non empty string Traceback (most recent call last):
  File "<ipython-input-19-3599527f68d1>", line 3, in <module>
    p.setname(100)
  File "<ipython-input-9-583d36012684>", line 12, in setname
    raise ValueError('name must be a non empty string')
ValueError: name must be a non empty string



In [20]:
import traceback
try:
    p.setname('')
except ValueError as ex:
    print(ex, traceback.format_exc())

name must be a non empty string Traceback (most recent call last):
  File "<ipython-input-20-252b25acab9b>", line 3, in <module>
    p.setname('')
  File "<ipython-input-9-583d36012684>", line 12, in setname
    raise ValueError('name must be a non empty string')
ValueError: name must be a non empty string



In [21]:
p.__dict__

{'_name': 'Alex'}

So this works, but it's a bit of pain to use the method names. So let's turn this into a property instead. We start with the class we just had and tweak it a bit:

In [35]:
class Person:
    def __init__(self,name):
        self._name = name
        
    def getname(self):
        print('getter called...')
        return self._name
    
    def setname(self,value):
        print('setter called..')
        if isinstance(value,str) and len(value.strip())>0:
            self._name = value
        else:
            raise ValueError('name must be a non empty string')
            
    name = property(fget=getname, fset=setname)

In [36]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'getname': <function __main__.Person.getname(self)>,
              'setname': <function __main__.Person.setname(self, value)>,
              'name': <property at 0x25f783b2638>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [37]:
p = Person('amit')

In [38]:
p.name

getter called...


'amit'

In [39]:
p.name = 'sumit'

setter called..


In [40]:
p.name

getter called...


'sumit'

In [41]:
p.__dict__

{'_name': 'sumit'}

In [42]:
try:
    p.name = None
except ValueError as ex:
    print(ex)

setter called..
name must be a non empty string


So now we have the benefit of using accessor methods, without having to call the methods explicitly.

In fact, even getattr and setattr will work the same way:



In [43]:
setattr(p, 'name', 'John')  # or p.name = 'John'

setter called..


In [44]:
getattr(p, 'name')  # or simply p.name

getter called...


'John'

In [45]:
p.__dict__

{'_name': 'John'}

You'll see we can find the underlying "private" attribute we are using to store the name. But the property itself (name) is not in the dictionary.



The property was defined in the class:

In [46]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'getname': <function __main__.Person.getname(self)>,
              'setname': <function __main__.Person.setname(self, value)>,
              'name': <property at 0x25f783b2638>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

So when we type p.name or p.name = value, Python recognizes that 'name is a property and therefore uses the accessor methods. (How it does we'll see later when we study descriptors).

What's interesting is that even if we muck around with the instance dictionary, Python does not get confused - (and as usual in Python, just because you can do something does not mean you should!)

In [47]:
p = Person('Alex')


In [48]:
p.name

getter called...


'Alex'

In [49]:
p.__dict__

{'_name': 'Alex'}

In [50]:
p.__dict__['name'] = 'John'


In [51]:
p.__dict__

{'_name': 'Alex', 'name': 'John'}

As you can see, we now have name in our instance dictionary.

Let's retrieve the name via dotted notation:

In [52]:
p.name

getter called...


'Alex'

That's obviously still using the getter method.

And setting the name:

In [53]:
p.name = 'Raymond'

setter called..


In [54]:
p.__dict__

{'_name': 'Raymond', 'name': 'John'}


As you can see, it used the setter method.

And the same thing happens with setattr and getattr:

In [55]:
getattr(p, 'name')

getter called...


'Raymond'

In [56]:
setattr(p, 'name', 'Python')


setter called..


In [57]:
p.__dict__


{'_name': 'Python', 'name': 'John'}

As you can see the setattr method used the property setter.

For completeness, let's see how the deleter method works:

In [59]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        print('getting name')
        return self._name
    
    def set_name(self, value):
        print('setting name')
        self._name = value
        
    def del_name(self):
        print('deleting name')
        del self._name  # or whatever "cleanup" we want to do
        
    name = property(fget=get_name, fset=set_name, fdel=del_name)

In [60]:
p = Person('Alex')


In [61]:
p.name

getting name


'Alex'

In [62]:
p.name='amit'

setting name


In [65]:
del p.name

deleting name


In [66]:
p.__dict__

{}

In [67]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'get_name': <function __main__.Person.get_name(self)>,
              'set_name': <function __main__.Person.set_name(self, value)>,
              'del_name': <function __main__.Person.del_name(self)>,
              'name': <property at 0x25f784161d8>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Now, the property still exists (since it is defined in the class) - all we did was remove the underlying value for the property (the _name instance attribute):

In [68]:
try:
    p.name
except AttributeError as ex:
    print(ex)

getting name
'Person' object has no attribute '_name'


In [69]:
p.name

getting name


AttributeError: 'Person' object has no attribute '_name'


As you can see the issue is that the getter function is trying to find _name in the attribute, which no longer exists. So the getter and setter still exist (i.e. the property still exists), so we can still assign to it:

In [70]:
p.name = 'amit'

setting name


In [71]:
p.__dict__

{'_name': 'amit'}

In [72]:
class Person:
    """This is a Person object"""
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
        
    name = property(get_name, set_name, doc="The person's name.")

In [73]:
p = Person('amit')

In [74]:
p.name

'amit'

In [75]:
help(Person.name)

Help on property:

    The person's name.



In [76]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Person(name)
 |  
 |  This is a Person object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_name(self)
 |  
 |  set_name(self, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  name
 |      The person's name.



### PropertyDecorator

As I explain in the lecture video, the property callable actually returns itself:

In [1]:
p = property(fget=lambda self: print('getting property'))

In [2]:
p

As you can see p is a property, and in fact is the same property that was created.

Think back to how decorators work:

In [3]:
def my_decorator(fn):
    print('decorating function')
    def inner(*args, **kwargs):
        print('running decorated function')
        return fn(*args, **kwargs)
    return inner

In [4]:
def undecorated_function(a, b):
    print('running original function')
    return a + b

In [5]:
undecorated_function = my_decorator(undecorated_function)

decorating function


In [6]:
undecorated_function(1,2)

running decorated function
running original function


3

And of course this is exactly what the decorator @ syntax does:

In [11]:
@my_decorator
def undecorated_function(a, b):
    print('running original function')
    return a + b

decorating function


In [12]:
undecorated_function(5,6)

running decorated function
running original function


11

The property callable creates a property object, and returns it.

In other words, we could create our property this way, as usual:

In [13]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def name(self):
        return self._name
    
    name = property(name)

In [14]:

p = Person('Alex')

p.name

'Alex'

But you'll notice that line: name = property(name) - that's exactly what the decorator syntax does for us!

In [15]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

In [16]:
p = Person('Guido')
p.name

'Guido'

If you refresh your memory on the single dispatch generic function decorator, you'll remember that the decorated function included another property, the register property that was itself a decorator.

Well, the property object has some properties, like setter that will basically accept a reference to the setter method, and return itself also.

In [19]:
p = property(lambda self: 'getter')
dir(p)

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']

So, we can "register" and setter method, using the setter callable, and get our property back as well:

In [20]:
p

In [21]:
p2 = p.setter(lambda self: 'setter')


In [24]:
hex(id(p)), hex(id(p2))

('0x22afd1f3638', '0x22afd1f3868')

Now you'll notice that the property id has changed. The setter callable actually creates a new property (with both the original getter, and the new setter assigned).

But that does not really matter, we just have a new property object that we can use to assign to a symbol - and that property will have both the getter and the setter defined.

Let's do this manually (without the decorator syntax first):

In [25]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def name(self):
        return self._name
    
    name = property(name)
    
    # creating another symbol that holds on to 
    # the name property
    name_prop = name 
    
    # because herte I'm redefining name, so we lose 
    # our original reference to the property object
    def name(self, value):
        self._name = value
        
    name = name_prop.setter(name)
    
    # finally delete name_prop which we no longer need
    del name_prop

In [26]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'name': <property at 0x22afd1fd1d8>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

nd we now have a name property that we created in two steps: first create the property with just a getter.

Then we replaced our property with a new property that had both the getter and the setter.

In [27]:
p = Person('Alex')
p.name

'Alex'

In [28]:
p.name = 'Raymond'
p.name

'Raymond'

Hopefully you can now see where the original property (with just the getter), had a callable setter that "added" the setter to the property (by creating a new property with both getter and setter), that also returned the (new) property object.

So, we can simplify our code this way:

In [29]:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # what's the property name now? --> name
    # so name has a setter callable
    @name.setter
    def name(self, value):
        self._name = value

Note that if we had not named our setter function name the property name would have changed!

Remember that:
```
@dec
def my_func():
    pass
    ```
returns a decorated function with the same name as the original function

In [30]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'name': <property at 0x22afd1fd1d8>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [31]:
p = Person('Alex')

In [32]:
p.name

'Alex'

In [33]:

p.name = 'Guido'
p.name

'Guido'

Just to show you, if we had not used the same name for the setter function:


In [34]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # property is now called name
    
    @name.setter
    def full_name(self, value):
        self._name = value

In [35]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'name': <property at 0x22afd1fdc78>,
              'full_name': <property at 0x22afd1fde08>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

As you can see we now have two properties on the class! The first one name will only work as a getter. And the second one full_name will work as both a getter and a setter:

In [36]:
p = Person('Alex')


In [38]:
p.name

'Alex'

In [39]:
p.full_name

'Alex'

In [40]:
p.full_name = 'Raymond'
p.full_name

'Raymond'

But this will not work

In [42]:
try:
    p.name = 'amit'
except AttributeError as e:
    print(e)

can't set attribute


Technically, the property callable has both a getter and setter method - so we can create the setter first, then "add in" the getter. But since the first argument to property is the getter, we have to work a bit more to do it:

In [43]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value

By the way, we now have a property that can be set, but not read back!

In [45]:

p = Person('Alex')

In [46]:
p.__dict__

{'_name': 'Alex'}

In [47]:
p.name = 'Raymond'

In [48]:
p.__dict__

{'_name': 'Raymond'}

In [50]:
try:
    p.name
except AttributeError as ex:
    print(ex)

unreadable attribute


So, if you ever need an attribute that is "write-only" - you can do it. Maybe the data is sensitive, and you want to set it, but not show back to users... But the data is never truly private, so at best you're obfuscating the data - so in my experience I've never had to do something like that. Just wanted you to see this in case the need ever came up.

But let's finish this up and make the property read/write:

In [51]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value
        
    @name.getter
    def name(self):
        return self._name

In [52]:
p = Person('amit')

In [53]:
p.name

'amit'

In [54]:
p.name = 'sumit'

In [55]:
p.name

'sumit'

The deleter works the same way, and we'll come back to it soon.

Lastly you'll recall that we could set up a docstring when using the property callable.

The standard technique is to simply define the docstring in the getter function:

In [58]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        """The Person's name."""
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

In [59]:
help(Person.name)

Help on property:

    The Person's name.



In [60]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Person(name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  name
 |      The Person's name.



What happens if we set it in the setter instead

In [64]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        """The Person's name."""
        self._name = value

In [65]:
help(Person.name)

Help on property:




In [66]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Person(name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  name



As you can see, the property docstring is only set on the getter. So how to set a docstring with a write-only property? We can do that when we create the initial property

In [67]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property(doc='Write-only name property.')
    
    @name.setter
    def name(self, value):
        self._name = value

In [68]:
help(Person.name)

Help on property:

    Write-only name property.

